In [2]:
import pandas as pd;
# PATHS
#agriculture (3)
agri1 = '../agriculture/data/final_data_bundesland_agriculture_animalcount.csv'
agri2 = '../agriculture/data/final_data_landkreis_agriculture_animalcount.csv'

#buildings (2)
bui1 = '../buildings/data/final_data_landkreis_newbuildings_renewables.csv'

#mobility (3)
mobi1 = '../mobility/data/final_data_landkreis_mobility_cartype.csv'

#waste (3)

#energy (3)


#combine all paths
paths = [mobi1,bui1,agri2]
paths

['../mobility/data/final_data_landkreis_mobility_cartype.csv',
 '../buildings/data/final_data_landkreis_newbuildings_renewables.csv',
 '../agriculture/data/final_data_landkreis_agriculture_animalcount.csv']

In [3]:
import json
import numpy as np
import os

#1. Create pipeline to fetch snippets of data for each location and indicator
#2. TODO: Define a hierarchy for indicators
#3. Create dynamic labels to differentiate datasets
#4. Check if regional data are available for each landkreis to store it as a parameter

def getDictionaryWithPrefix(df):
    #set index
    df.set_index('AGS',inplace=True)

    #fill nan with jsonable format
    df = df.fillna(np.nan).replace([np.nan], [None])
    
    #drop name column
    df.drop(columns=["Name"],axis=1,inplace=True)

    #get dynamic labels to differentiate datasets
    #get prefix
    columns = df.columns.tolist()
    prefix = os.path.commonprefix(columns)
    prefix

    #TODO: create hiearchy from prefixes

    #remove prefix from column names
    df.columns = df.columns.str.removeprefix(prefix)

    #create dict from dataframe
    data_dict = df.T.to_dict('dict')
    
    return prefix,data_dict 


def generate_json(paths):

    landkreisData = {}

    indicators = []
    dicts = []

    #pipeline to fetch snippets of data for each location and indicator
    for path in paths:
        #read dataset
        df = pd.read_csv(path)

        prefix,data_dict = getDictionaryWithPrefix(df)
        indicators.append(prefix)
        dicts.append(data_dict)

    print("indicator: ",indicators)

    #get ags list
    df = pd.read_csv(mobi1)
    ags = df.AGS.unique().tolist()
    #ags = [2,5] #debugging

    for ort in ags:

        ort = int(ort)

        #create top level for each location
        landkreisData[str(ort)] = {
            'id':ort,
            'score':2}

        data = {}
        
        #iterate over indicators
        for i,ind in enumerate(indicators):

            if(ort in dicts[i]):
                
   
                local_data = [{"key":index ,"year":item[0][-4:],"column":item[0][:-5],"value":item[1]} for index,item in enumerate(dicts[i][ort].items())]
        
                #create one entry per indicator
                #Check if regional data are available for each landkreis to store it as a parameter
                data[ind] = {
                    'benchmark':200,
                    'regional': True if ort > 17 else False,
                    'unit': 'cars / 1000 residents',
                    'data':local_data
                }
            
            else:
                #TODO: add bundesland level to all indicators, so this is only false for landkreise
                data[ind] = {
                    'regional':False,
                }

            #add data entry to location level
            #landkreisData[str(ort)][ind] = "added data"
            landkreisData[str(ort)][ind] = data[ind]
        

    with open('data-inprogress.json', 'w', encoding='utf-8') as f:
        json.dump(landkreisData, f, ensure_ascii=False, indent=4)


generate_json(paths)

indicator:  ['mobility_cartype_density_', 'newbuilding_energy_', 'animalcount_Anzahl_']
